In [4]:
import pandas as pd
from pandasql import sqldf

# Read the Excel file into separate DataFrames
file_path = r"C:\Users\vivek.p\Downloads\ServiceInsights_POC_230810.xlsx"
cases_df = pd.read_excel(file_path, sheet_name="Cases")
Sites_df = pd.read_excel(file_path, sheet_name="Sites")
service_appointments_df = pd.read_excel(file_path, sheet_name="ServiceAppointments")
Orders_df = pd.read_excel(file_path, sheet_name="Orders")
SpareParts_df = pd.read_excel(file_path, sheet_name="SpareParts")
SoftwareLicenses_df = pd.read_excel(file_path, sheet_name="SoftwareLicenses")


# Define the SQL query using pandasql
pysqldf = lambda q: sqldf(q, globals())

In [13]:
# Perform SQL-like operations between the DataFrames
from datetime import datetime, timedelta

# Define a helper function for date calculations
def calculate_order_month(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    last_day_of_previous_month = date_obj.replace(day=1) - timedelta(days=1)
    order_month = last_day_of_previous_month.replace(day=1)
    return order_month.strftime('%Y-%m-%d')

# Perform SQL-like operations between the DataFrames
query = """
SELECT SiteName as 'Site',
	 '{}' as 'Order Month',
	 COUNT(*) as 'Number of Orders'
FROM Orders_df o
INNER JOIN Sites_df s on o.SiteId = s.SiteId
WHERE Date BETWEEN '2023-01-01' and '2023-08-31'
GROUP BY SiteName, '{}'
ORDER BY 'Order Month', 'Site'
""".format(calculate_order_month('2023-08-31'), calculate_order_month('2023-08-31'))
result_df = pysqldf(query)

# Write the result DataFrame to a new Excel file
result_file_path = "1.xlsx"
result_df.to_excel(result_file_path, index=False)

In [16]:
# Define a helper function for mapping status based on conditions
def map_status(row):
    today = pd.Timestamp.now().date()
    if row["StartDate"].date() <= today <= row["EndDate"].date():
        if pd.isnull(row["Resource"]):
            return "Review Required"
        else:
            return "In Progress"
    elif today > row["EndDate"].date():
        return "Completed"
    elif today < row["StartDate"].date():
        return "Upcoming"
    elif pd.isnull(row["StartDate"]) or pd.isnull(row["EndDate"]):
        if today < (row["DueDate"] - pd.Timedelta(days=90)).date():
            return "Nothing"
        elif today < (row["DueDate"] - pd.Timedelta(days=14)).date():
            return "Scheduling Required"
        else:
            return "Review Required"
    else:
        return row["Status"]

# Apply the mapping function to the DataFrame
service_appointments_df["Action"] = service_appointments_df.apply(map_status, axis=1)

# Perform SQL-like operations between the DataFrames
query = """
SELECT SiteName,
       Action,
       COUNT(*) as 'Number of Appointments'
FROM service_appointments_df fsa
INNER JOIN Sites_df s on fsa.SiteID = s.SiteId
WHERE StartDate BETWEEN '2023-01-01' AND '2023-12-31'
      OR EndDate BETWEEN '2023-01-01' AND '2023-12-31'
GROUP BY SiteName, Action
ORDER BY SiteName, Action;
"""

result_df = pysqldf(query)

# Write the result DataFrame to a new Excel file
result_file_path = "3.xlsx"
result_df.to_excel(result_file_path, index=False)
